In [1]:
import numpy as np
import pandas as pd
import scipy.io as io
import os

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

torch.cuda.is_available()

True

In [3]:
import timm
from torch.utils.data.sampler import SubsetRandomSampler, WeightedRandomSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


In [4]:
import wandb
# WANDB_NOTEBOOK_NAME = 'Train_Analysis_VRB_wandb_sweep.ipynb'
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xiaosuhu86. Use `wandb login --relogin` to force relogin


True

In [5]:
sweep_configuration = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {'goal': 'minimize', 'name': 'val_loss'},
    'parameters': 
    {
        'batch_size': {'values': [32, 64]},
        'epochs': {'values': [5, 10, 15]},
        'lr': {'values': [0.001, 0.0005, 0.0001]},
        'model': {'values': ['efficientnet_b0','efficientnet_b2','efficientnet_b4','resnet18d','resnet34d','resnet50d','resnet101d']}
     }
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project='PreBreathTraining-Pytorch-test')


500 response executing GraphQL.
{"errors":[{"message":"An internal error occurred. Please contact support.","path":["upsertSweep"]}],"data":{"upsertSweep":null}}
wandb: ERROR Error while calling W&B API: An internal error occurred. Please contact support. (<Response [500]>)


Create sweep with ID: knt59wpc
Sweep URL: https://wandb.ai/xiaosuhu86/TMB-Pytorch-test/sweeps/knt59wpc


In [6]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [7]:
class FNIRS_Dataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.targets = self.img_labels['Label']

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 4])
        image = np.float32(io.loadmat(img_path)['fnirsimg']).reshape(3,21,45)
        label = self.img_labels.iloc[idx, 5]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    return loss

def validate(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    model.eval()     # Optional when not using Model Specific layer

    total_correct = 0
    total_instances = 0
    y_true=[]
    y_pred=[]

    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred,y)
        classifications = torch.argmax(model(X), dim=1)
        correct_predictions = sum(classifications==y).item()

        total_correct+=correct_predictions
        total_instances+=len(y)

        y_pred.extend(classifications.data.cpu().numpy())
        y_true.extend(y.data.cpu().numpy())
            
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"val_loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    
    acc = total_correct/total_instances

    # constant for classes
    classes = ('No_Pain','Pain')
    cf_matrix = confusion_matrix(y_true, y_pred)
    df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
                     columns = [i for i in classes])
    
    TP = df_cm['Pain']['Pain']
    TN = df_cm['No_Pain']['No_Pain']

    print(f"val_acc: {acc:>7f} TP: {TP:>4f} TN: {TN:>4f}")

    return loss, acc, TP, TN

In [9]:
# Load the data with imbalanced weights
train_data=FNIRS_Dataset(
    '../Label_PreTraining.csv',
    '../PreTrainingdata/'
)

total_targets=torch.asarray(train_data.targets)
train_idx, valid_idx= train_test_split(
    np.arange(len(total_targets)), test_size=0.2, random_state=42, shuffle=True, stratify=total_targets)

train_set = torch.utils.data.Subset(train_data, train_idx)
# val_set = torch.utils.data.Subset(train_data, valid_idx)

train_sample_count = torch.tensor(
    [(total_targets[train_idx] == t).sum() for t in torch.unique(total_targets, sorted=True)])
train_weight = 1. / train_sample_count.float()
train_sample_weight = torch.tensor([train_weight[t] for t in total_targets[train_idx]])

#Creating PT data samplers
train_sampler = WeightedRandomSampler(train_sample_weight, len(train_sample_weight))
valid_sampler = SubsetRandomSampler(valid_idx)

def main():
    run = wandb.init()

    batch_size = wandb.config.batch_size

    # Create data loaders:
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, 
                                            sampler=train_sampler)
    validation_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                                    sampler=valid_sampler)

    # Model defining
    model = timm.create_model(wandb.config.model, num_classes=2, pretrained=False)
    model=model.to(device)
    
    # Other learning parameters
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.lr)
    epochs = wandb.config.epochs

    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_loss = train(train_loader, model, loss_fn, optimizer)
        val_loss, val_acc, TP, TN = validate(validation_loader, model, loss_fn)

        wandb.log({
        'epoch': t+1, 
        #'train_acc': train_acc,
        'train_loss': train_loss, 
        'val_acc': val_acc, 
        'val_loss': val_loss,
        'true_pos': TP,
        'true_neg': TN,
        'model': wandb.config.model
        })

wandb.agent(sweep_id, function=main)

wandb: Agent Starting Run: a7dotcmd with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	lr: 0.001
wandb: 	model: efficientnet_b0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1
-------------------------------
loss: 2.761942  [   32/12182]
loss: 1.170739  [ 3232/12182]
loss: 1.129218  [ 6432/12182]
loss: 0.848713  [ 9632/12182]
val_loss: 1.414662  [   32/15228]
val_acc: 0.771832 TP: 0.159686 TN: 0.859610
Epoch 2
-------------------------------
loss: 0.819852  [   32/12182]
loss: 0.764893  [ 3232/12182]
loss: 0.764489  [ 6432/12182]
loss: 0.712384  [ 9632/12182]
val_loss: 1.128606  [   32/15228]
val_acc: 0.814183 TP: 0.081152 TN: 0.919294
Epoch 3
-------------------------------
loss: 0.667757  [   32/12182]
loss: 0.680704  [ 3232/12182]
loss: 0.615587  [ 6432/12182]
loss: 0.480297  [ 9632/12182]
val_loss: 2.546973  [   32/15228]
val_acc: 0.677938 TP: 0.340314 TN: 0.726351
Epoch 4
-------------------------------
loss: 0.569769  [   32/12182]
loss: 0.543710  [ 3232/12182]
loss: 0.664078  [ 6432/12182]
loss: 0.406911  [ 9632/12182]
val_loss: 0.778536  [   32/15228]
val_acc: 0.636901 TP: 0.458115 TN: 0.662538
Epoch 5
-------------------------------
loss: 0.

epoch,▁▃▅▆█
train_loss,▄█▄▂▁
true_neg,▆█▃▁▆
true_pos,▂▁▆█▃
val_acc,▆█▃▁▆
val_loss,▅▁▁▁█
epoch,5
model,efficientnet_b0
train_loss,0.39697
true_neg,0.86036
true_pos,0.17539


wandb: Agent Starting Run: u28rdztu with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	lr: 0.001
wandb: 	model: efficientnet_b2
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1
-------------------------------
loss: 5.384334  [   32/12182]
loss: 1.101776  [ 3232/12182]
loss: 0.738829  [ 6432/12182]
loss: 0.620947  [ 9632/12182]
val_loss: 1.075847  [   32/15228]
val_acc: 0.477347 TP: 0.617801 TN: 0.457207
Epoch 2
-------------------------------
loss: 0.720139  [   32/12182]
loss: 0.662564  [ 3232/12182]
loss: 0.812056  [ 6432/12182]
loss: 0.751167  [ 9632/12182]
val_loss: 0.715576  [   32/15228]
val_acc: 0.565988 TP: 0.494764 TN: 0.576201
Epoch 3
-------------------------------
loss: 0.618050  [   32/12182]
loss: 0.713429  [ 3232/12182]
loss: 0.733845  [ 6432/12182]
loss: 0.828566  [ 9632/12182]
val_loss: 0.501902  [   32/15228]
val_acc: 0.779054 TP: 0.141361 TN: 0.870495
Epoch 4
-------------------------------
loss: 0.633133  [   32/12182]
loss: 0.557306  [ 3232/12182]
loss: 0.422534  [ 6432/12182]
loss: 0.508544  [ 9632/12182]
val_loss: 0.671159  [   32/15228]
val_acc: 0.761655 TP: 0.185864 TN: 0.844219
Epoch 5
-------------------------------
loss: 0.

...

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_loss,█▃▂▁▁▃▁▁▁▁▁▁▁▁▂
true_neg,▁▄▇█▆▇▆▇▇▇▇▇███
true_pos,█▆▂▁▃▃▃▁▃▂▃▂▂▁▁
val_acc,▁▄▇█▆▇▆▇▇▇▇▇███
val_loss,▂▃▅▃▁▁▃▇▃▆▅▆█▆▂
epoch,15
model,resnet50d
train_loss,0.05301
true_neg,0.97372
true_pos,0.06021


wandb: Agent Starting Run: bcwmc93g with config:
wandb: 	batch_size: 64
wandb: 	epochs: 15
wandb: 	lr: 0.0001
wandb: 	model: resnet101d
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1
-------------------------------
loss: 0.717293  [   64/12182]
loss: 0.620869  [ 6464/12182]
val_loss: 0.545725  [   64/15228]
val_acc: 0.748523 TP: 0.248691 TN: 0.820195
Epoch 2
-------------------------------
loss: 0.426268  [   64/12182]
loss: 0.198704  [ 6464/12182]
val_loss: 0.677308  [   64/15228]
val_acc: 0.719304 TP: 0.392670 TN: 0.766141
Epoch 3
-------------------------------
loss: 0.168628  [   64/12182]
loss: 0.047352  [ 6464/12182]
val_loss: 0.982267  [   64/15228]
val_acc: 0.704531 TP: 0.437173 TN: 0.742868
Epoch 4
-------------------------------
loss: 0.100421  [   64/12182]
loss: 0.021646  [ 6464/12182]
val_loss: 1.066227  [   64/15228]
val_acc: 0.866382 TP: 0.041885 TN: 0.984610
Epoch 5
-------------------------------
loss: 0.156461  [   64/12182]
loss: 0.069998  [ 6464/12182]
val_loss: 0.862023  [   64/15228]
val_acc: 0.836179 TP: 0.125654 TN: 0.938063
Epoch 6
-------------------------------
loss: 0.019489  [   64/12182]
loss: 0.132046  [ 6464/12182]
val_loss: 

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_loss,█▃▁▂▂▁▁▁▁▁▂▁▁▁▂
true_neg,▃▂▁█▇▇▇▇▇▅▆▃█▅▅
true_pos,▅▇█▁▂▂▃▂▃▄▃▅▂▃▄
val_acc,▃▂▁█▇▇▇▇▇▅▆▃█▅▆
val_loss,▂▂▃▄▃▄▂▄█▃▁▄▅▅▅
epoch,15
model,resnet101d
train_loss,0.03338
true_neg,0.89715
true_pos,0.20942


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [ ]:
# Load the data with imbalanced weights
train_data=FNIRS_Dataset(
    '../Label_TMB.csv',
    '../TMBdata/'
)

total_targets=torch.asarray(train_data.targets)
train_idx, valid_idx= train_test_split(
    np.arange(len(total_targets)), test_size=0.2, random_state=42, shuffle=True, stratify=total_targets)

train_set = torch.utils.data.Subset(train_data, train_idx)
# val_set = torch.utils.data.Subset(train_data, valid_idx)

train_sample_count = torch.tensor(
    [(total_targets[train_idx] == t).sum() for t in torch.unique(total_targets, sorted=True)])
train_weight = 1. / train_sample_count.float()
train_sample_weight = torch.tensor([train_weight[t] for t in total_targets[train_idx]])

#Creating PT data samplers
train_sampler = WeightedRandomSampler(train_sample_weight, len(train_sample_weight))
valid_sampler = SubsetRandomSampler(valid_idx)

def main():
    run = wandb.init()

    batch_size = wandb.config.batch_size

    # Create data loaders:
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, 
                                            sampler=train_sampler)
    validation_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                                    sampler=valid_sampler)

    # Model defining
    model = timm.create_model(wandb.config.model, num_classes=2, pretrained=False)
    model=model.to(device)
    
    # Other learning parameters
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.lr)
    epochs = wandb.config.epochs

    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_loss = train(train_loader, model, loss_fn, optimizer)
        val_loss, val_acc, TP, TN = validate(validation_loader, model, loss_fn)

        wandb.log({
        'epoch': t+1, 
        #'train_acc': train_acc,
        'train_loss': train_loss, 
        'val_acc': val_acc, 
        'val_loss': val_loss,
        'true_pos': TP,
        'true_neg': TN,
        'model': wandb.config.model
        })

wandb.agent(sweep_id, function=main)

wandb: Agent Starting Run: a7dotcmd with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	lr: 0.001
wandb: 	model: efficientnet_b0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1
-------------------------------
loss: 2.761942  [   32/12182]
loss: 1.170739  [ 3232/12182]
loss: 1.129218  [ 6432/12182]
loss: 0.848713  [ 9632/12182]
val_loss: 1.414662  [   32/15228]
val_acc: 0.771832 TP: 0.159686 TN: 0.859610
Epoch 2
-------------------------------
loss: 0.819852  [   32/12182]
loss: 0.764893  [ 3232/12182]
loss: 0.764489  [ 6432/12182]
loss: 0.712384  [ 9632/12182]
val_loss: 1.128606  [   32/15228]
val_acc: 0.814183 TP: 0.081152 TN: 0.919294
Epoch 3
-------------------------------
loss: 0.667757  [   32/12182]
loss: 0.680704  [ 3232/12182]
loss: 0.615587  [ 6432/12182]
loss: 0.480297  [ 9632/12182]
val_loss: 2.546973  [   32/15228]
val_acc: 0.677938 TP: 0.340314 TN: 0.726351
Epoch 4
-------------------------------
loss: 0.569769  [   32/12182]
loss: 0.543710  [ 3232/12182]
loss: 0.664078  [ 6432/12182]
loss: 0.406911  [ 9632/12182]
val_loss: 0.778536  [   32/15228]
val_acc: 0.636901 TP: 0.458115 TN: 0.662538
Epoch 5
-------------------------------
loss: 0.

epoch,▁▃▅▆█
train_loss,▄█▄▂▁
true_neg,▆█▃▁▆
true_pos,▂▁▆█▃
val_acc,▆█▃▁▆
val_loss,▅▁▁▁█
epoch,5
model,efficientnet_b0
train_loss,0.39697
true_neg,0.86036
true_pos,0.17539


wandb: Agent Starting Run: u28rdztu with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	lr: 0.001
wandb: 	model: efficientnet_b2
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1
-------------------------------
loss: 5.384334  [   32/12182]
loss: 1.101776  [ 3232/12182]
loss: 0.738829  [ 6432/12182]
loss: 0.620947  [ 9632/12182]
val_loss: 1.075847  [   32/15228]
val_acc: 0.477347 TP: 0.617801 TN: 0.457207
Epoch 2
-------------------------------
loss: 0.720139  [   32/12182]
loss: 0.662564  [ 3232/12182]
loss: 0.812056  [ 6432/12182]
loss: 0.751167  [ 9632/12182]
val_loss: 0.715576  [   32/15228]
val_acc: 0.565988 TP: 0.494764 TN: 0.576201
Epoch 3
-------------------------------
loss: 0.618050  [   32/12182]
loss: 0.713429  [ 3232/12182]
loss: 0.733845  [ 6432/12182]
loss: 0.828566  [ 9632/12182]
val_loss: 0.501902  [   32/15228]
val_acc: 0.779054 TP: 0.141361 TN: 0.870495
Epoch 4
-------------------------------
loss: 0.633133  [   32/12182]
loss: 0.557306  [ 3232/12182]
loss: 0.422534  [ 6432/12182]
loss: 0.508544  [ 9632/12182]
val_loss: 0.671159  [   32/15228]
val_acc: 0.761655 TP: 0.185864 TN: 0.844219
Epoch 5
-------------------------------
loss: 0.

...

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_loss,█▃▂▁▁▃▁▁▁▁▁▁▁▁▂
true_neg,▁▄▇█▆▇▆▇▇▇▇▇███
true_pos,█▆▂▁▃▃▃▁▃▂▃▂▂▁▁
val_acc,▁▄▇█▆▇▆▇▇▇▇▇███
val_loss,▂▃▅▃▁▁▃▇▃▆▅▆█▆▂
epoch,15
model,resnet50d
train_loss,0.05301
true_neg,0.97372
true_pos,0.06021


wandb: Agent Starting Run: bcwmc93g with config:
wandb: 	batch_size: 64
wandb: 	epochs: 15
wandb: 	lr: 0.0001
wandb: 	model: resnet101d
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1
-------------------------------
loss: 0.717293  [   64/12182]
loss: 0.620869  [ 6464/12182]
val_loss: 0.545725  [   64/15228]
val_acc: 0.748523 TP: 0.248691 TN: 0.820195
Epoch 2
-------------------------------
loss: 0.426268  [   64/12182]
loss: 0.198704  [ 6464/12182]
val_loss: 0.677308  [   64/15228]
val_acc: 0.719304 TP: 0.392670 TN: 0.766141
Epoch 3
-------------------------------
loss: 0.168628  [   64/12182]
loss: 0.047352  [ 6464/12182]
val_loss: 0.982267  [   64/15228]
val_acc: 0.704531 TP: 0.437173 TN: 0.742868
Epoch 4
-------------------------------
loss: 0.100421  [   64/12182]
loss: 0.021646  [ 6464/12182]
val_loss: 1.066227  [   64/15228]
val_acc: 0.866382 TP: 0.041885 TN: 0.984610
Epoch 5
-------------------------------
loss: 0.156461  [   64/12182]
loss: 0.069998  [ 6464/12182]
val_loss: 0.862023  [   64/15228]
val_acc: 0.836179 TP: 0.125654 TN: 0.938063
Epoch 6
-------------------------------
loss: 0.019489  [   64/12182]
loss: 0.132046  [ 6464/12182]
val_loss: 

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_loss,█▃▁▂▂▁▁▁▁▁▂▁▁▁▂
true_neg,▃▂▁█▇▇▇▇▇▅▆▃█▅▅
true_pos,▅▇█▁▂▂▃▂▃▄▃▅▂▃▄
val_acc,▃▂▁█▇▇▇▇▇▅▆▃█▅▆
val_loss,▂▂▃▄▃▄▂▄█▃▁▄▅▅▅
epoch,15
model,resnet101d
train_loss,0.03338
true_neg,0.89715
true_pos,0.20942


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
